In [1]:
import pandas as pd

In [2]:
import gspread

In [3]:
from oauth2client.service_account import ServiceAccountCredentials

In [4]:
from df2gspread import df2gspread as d2g

In [13]:
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('./Jupyter-GoogleDrive-1587a93c5fd8.json', scope)
gc = gspread.authorize(credentials)

ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [10]:
spreadsheet_key = '1E1RsEUD0oegbZRUMLwBARluOuQzeMNO3DRnEIHeHCuM' 
book = gc.open_by_key(spreadsheet_key) 

NameError: name 'gc' is not defined

In [ ]:
worksheet = book.worksheet("initial QC") 
raw_table = worksheet.get_all_values()

In [7]:
atac_df = pd.DataFrame(raw_table[1:], columns=["sample_run", "length", "m_seq"])

In [8]:
atac_df.head()

,sample_run,length,m_seq
0,IJ006_S56_L005_R1_001,51 bp,29.7
1,IJ007_S68_L006_R1_001,51 bp,23.7
2,IJ015_S60_L005_R1_001,51 bp,56.4
3,IJ016_S58_L005_R1_001,51 bp,37.1
4,IJ017_S57_L005_R1_001,51 bp,29.1


In [9]:
atac_df["m_seq"] = atac_df["m_seq"].astype("float")
low_qual = atac_df[atac_df["m_seq"] <= 0.5]

In [10]:
low_qual_list = low_qual["sample_run"].tolist()

In [11]:
print(low_qual_list)

['LM101_S1_L001_R1_001', 'LM101_S1_L001_R2_001', 'LM107_S4_L001_R1_001', 'LM107_S4_L001_R2_001', 'LMA036_S78_L001_R1_001', 'LMA036_S78_L001_R2_001', 'LMA037_S79_L001_R1_001', 'LMA037_S79_L001_R2_001', 'LMA038_S80_L001_R1_001', 'LMA038_S80_L001_R2_001', 'LMA039_S81_L001_R1_001', 'LMA039_S81_L001_R2_001', 'LMA040_S82_L001_R1_001', 'LMA040_S82_L001_R2_001', 'LMA041_S83_L001_R1_001', 'LMA041_S83_L001_R2_001', 'LMA042_S84_L001_R1_001', 'LMA042_S84_L001_R2_001', 'LMA043_S85_L001_R1_001', 'LMA043_S85_L001_R2_001']


In [12]:
wks_name = 'low_qual'

d2g.upload(low_qual, spreadsheet_key, wks_name, credentials=credentials, row_names=True)

<Worksheet 'low_qual' id:2041424079>

In [13]:
print ("[{0}]".format(' '.join(map(str, low_qual_list))))

[LM101_S1_L001_R1_001 LM101_S1_L001_R2_001 LM107_S4_L001_R1_001 LM107_S4_L001_R2_001 LMA036_S78_L001_R1_001 LMA036_S78_L001_R2_001 LMA037_S79_L001_R1_001 LMA037_S79_L001_R2_001 LMA038_S80_L001_R1_001 LMA038_S80_L001_R2_001 LMA039_S81_L001_R1_001 LMA039_S81_L001_R2_001 LMA040_S82_L001_R1_001 LMA040_S82_L001_R2_001 LMA041_S83_L001_R1_001 LMA041_S83_L001_R2_001 LMA042_S84_L001_R1_001 LMA042_S84_L001_R2_001 LMA043_S85_L001_R1_001 LMA043_S85_L001_R2_001]


In [14]:
print (len(low_qual_list))

20


## Add merged data

In [37]:
merged = book.worksheet("merged") 
merged_table = merged.get_all_values()
merged_table_df = pd.DataFrame(merged_table,columns=["sample_run", "length", "m_seq"])
merged_table_df["sample_name"] = merged_table_df["sample_run"].str.extract(r'(.+?)_')
merged_table_df["end"] = merged_table_df["sample_run"].str.extract(r'.*(R[12])')

In [38]:
print (merged_table_df.shape[0])

54


In [39]:
wks_name = 'merged'

d2g.upload(merged_table_df, spreadsheet_key, wks_name, credentials=credentials, row_names=True)

<Worksheet 'merged' id:1547344415>

In [42]:
print (set(merged_table_df["sample_name"].tolist()), len(set(merged_table_df["sample_name"].tolist())))

{'LM109', 'LM086', 'LM117', 'LM116', 'IJ017', 'LMA043', 'IJ007', 'LMA036', 'LM111', 'LM115', 'LM061', 'LM064', 'LM113', 'LMA038', 'LMA041', 'LM063', 'LM084', 'LM035', 'LM034', 'LM101', 'LM062', 'LM059', 'IJ006', 'LMA040', 'LMA042', 'LMA037', 'IJ015', 'LM110', 'LMA039', 'LM085', 'LM112', 'LM114', 'IJ016', 'LM107', 'IJ005'} 35


In [54]:
merged_table_df_R2 = merged_table_df[merged_table_df["end"] == "R2"]
print(merged_table_df_R2.shape[0])

19


In [40]:
merged_101 = merged_table_df[merged_table_df["length"] == "101 bp"]

In [41]:
merged_101.head()

,sample_run,length,m_seq,sample_name,end
16,LM101_R1,101 bp,46.4,LM101,R1
17,LM101_R2,101 bp,46.4,LM101,R2
18,LM107_R1,101 bp,49.5,LM107,R1
19,LM107_R2,101 bp,49.5,LM107,R2
20,LM109_R1,101 bp,82.7,LM109,R1


In [43]:
print (merged_101['sample_run'].tolist())
print (len(merged_101['sample_run'].tolist()))

['LM101_R1', 'LM101_R2', 'LM107_R1', 'LM107_R2', 'LM109_R1', 'LM109_R2', 'LM110_R1', 'LM110_R2', 'LM111_R1', 'LM111_R2', 'LM112_R1', 'LM112_R2', 'LM113_R1', 'LM113_R2', 'LM114_R1', 'LM114_R2', 'LM115_R1', 'LM115_R2', 'LM116_R1', 'LM116_R2', 'LM117_R1', 'LM117_R2']
22


In [44]:
merged_151 = merged_table_df[merged_table_df["length"] == "151 bp"]

In [45]:
merged_151.head()

,sample_run,length,m_seq,sample_name,end
38,LMA036_R1,151 bp,47.2,LMA036,R1
39,LMA036_R2,151 bp,47.2,LMA036,R2
40,LMA037_R1,151 bp,45.5,LMA037,R1
41,LMA037_R2,151 bp,45.5,LMA037,R2
42,LMA038_R1,151 bp,51.8,LMA038,R1


In [46]:
print (merged_151['sample_run'].tolist())

['LMA036_R1', 'LMA036_R2', 'LMA037_R1', 'LMA037_R2', 'LMA038_R1', 'LMA038_R2', 'LMA039_R1', 'LMA039_R2', 'LMA040_R1', 'LMA040_R2', 'LMA041_R1', 'LMA041_R2', 'LMA042_R1', 'LMA042_R2', 'LMA043_R1', 'LMA043_R2']


In [47]:
print (len(merged_151['sample_run'].tolist()))

16


In [48]:
merged_51 = merged_table_df[merged_table_df["length"] == "51 bp"]

In [49]:
print (" ".join(merged_51["sample_name"].tolist()))

IJ005 IJ006 IJ007 IJ015 IJ016 IJ017 LM034 LM035 LM059 LM061 LM062 LM063 LM064 LM084 LM085 LM086


In [50]:
print(len(merged_51["sample_name"].tolist()))

16


```bash
for i in IJ005 IJ006 IJ007 IJ015 IJ016 IJ017 LM034 LM035 LM059 LM061 LM062 LM063 LM064 LM084 LM085 LM086; do rename fastq trim.fastq $i*; done
```

## Add trimmed data

In [23]:
worksheet = book.worksheet("Cutadapt") 
trim_table = worksheet.get_all_values()

NameError: name 'book' is not defined

In [22]:
trimmed_table_df = pd.DataFrame(trim_table,columns=["sample_run", "length", "m_seq"])
trimmed_table_df["sample_name"] = trimmed_table_df["sample_run"].str.extract(r'(.+?)_')
trimmed_table_df["end"] = trimmed_table_df["sample_run"].str.extract(r'.*(R[12])')

NameError: name 'trim_table' is not defined